<a href="https://colab.research.google.com/github/blackhawkee/llm-play/blob/main/stable_diffusion_xl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U autotrain-advanced
!autotrain setup --update-torch

  Using cached triton-2.0.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.3 MB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_nccl_cu11-2.14.3-py3-none-manylinux1_x86_64.whl (177.1 MB)
  Using cached nvidia_nvtx_cu11-11.7.91-py3-none-manylinux1_x86_64.whl (98 kB)
  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached 

In [ ]:
import requests
import os
images = ["https://c4.wallpaperflare.com/wallpaper/158/382/512/5bd34deb802eb-wallpaper-preview.jpg",
            "https://c1.wallpaperflare.com/preview/51/466/512/dog-mascot-labrador-animal.jpg",
            "https://cdn.pixabay.com/photo/2016/02/19/15/46/labrador-retriever-1210559_1280.jpg",
            "https://c1.wallpaperflare.com/preview/512/113/145/dog-cute-pet-canidae.jpg"
            ]

images_directory = "/content/images"
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

for idx, url in enumerate(images):
  response = requests.get(url)
  with open(f"{images_directory}/image_{str(idx)}.jpg", "wb") as f:
      f.write(response.content)

In [ ]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output output/ \
--image-path /content/images/ \
--prompt "photo of sks dog" \
--resolution 512 \
--batch-size 1 \
--num-steps 200 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--use-8bit-adam

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
2023-07-28 11:49:30.178679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
> INFO    Namespace(version=False, model='stabilityai/stable-diffusion-xl-base-1.0', revision=None, tokenizer=None, image_path='/content/images/', class_image_path=None, prompt='photo of sks dog', class_prompt=None, num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, output='output/', seed=42, resolution=512, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epoch

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype = torch.float16
)
# pipe.to("cuda")
pipe.load_lora_weights(
    "output/", weight_name = "pytorch_lora_weights.safetensors"
)

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype = torch.float16
)
# refiner.to("cuda")

prompt = "portrait of sks dog, cartoon, pixar, 3d, headshots, fantasy, 4k, uhd"
for seed in range(30):
  generator = torch.Generator("cuda").manual_seed(seed)
  image = pipe(
      prompt=prompt, generator=generator, num_inference_steps=25
  )
  image = image.images[0]
  image = refiner(prompt=prompt, generator=generator, image=image)
  image=image.images[0]
  image.save(f"images/{seed}.png")